In [1]:
with open('inputs/16.txt', 'r') as file:
    inp = file.read()[:-1]

In [2]:
def to_binary(inp):
    scale = 16 ## equals to hexadecimal
    num_of_bits = len(inp)*4
    return bin(int(inp, scale))[2:].zfill(num_of_bits)

In [3]:
def literal(inp, pos):
    number = ""
    i = 0
    while True:   
        number += inp[pos+7+i:pos+11+i]
        if int(inp[pos+6+i]) != 1:
            break
        i += 5      
    return int(number, 2), 11+i+pos

In [4]:
def pv_and_type(inp, pos):
    packet_version = inp[pos:3+pos]
    type_ID = inp[pos+3:6+pos]
    return int(packet_version, 2), int(type_ID, 2)

In [5]:
# Task 1 Version number
def packet1(inp, pos=0, max_pos=0, nr_sub=1, nr_current=1, v=0):
    #print(inp[pos:], max_pos, nr_sub)
    pv, typ = pv_and_type(inp, pos)
    #print(pv, typ)
    if typ == 4:
        number, pos = literal(inp, pos)
        #print(number)
        if pos < max_pos:
            pos, v = packet1(inp, pos, max_pos, v=v)
        if nr_current < nr_sub:
            pos, v = packet1(inp, pos, max_pos=0, nr_sub=nr_sub, nr_current=nr_current+1, v=v)
        
    elif int(inp[pos+6]) == 0:
        length = int(inp[pos+7:pos+22], 2)
        pos += 22
        new_max_pos = pos + length
        pos, v = packet1(inp, pos, new_max_pos, v=v)
        if pos < max_pos:
            pos, v = packet1(inp, pos, max_pos, v=v)
        if nr_current < nr_sub:
            pos, v = packet1(inp, pos, max_pos=0, nr_sub=nr_sub, nr_current=nr_current+1, v=v)
    else:
        new_nr_sub = int(inp[pos+7:pos+18], 2)
        pos += 18
        pos, v = packet1(inp, pos, max_pos=0, nr_sub=new_nr_sub, v=v)
        if pos < max_pos:
            pos, v = packet1(inp, pos, max_pos, v=v)
        if nr_current < nr_sub:
            pos, v = packet1(inp, pos, max_pos=0, nr_sub=nr_sub, nr_current=nr_current+1, v=v)
    
    return pos, v+pv

In [6]:
# Task 2 operator
def packet2(inp, pos=0, max_pos=0, nr_sub=1, nr_current=1, v=[]):
    pv, typ = pv_and_type(inp, pos)
    if typ == 4:
        number, pos = literal(inp, pos)
        v += [number]
    else:
        s = [i for i in v]
        if int(inp[pos+6]) == 0:
            length = int(inp[pos+7:pos+22], 2)
            pos += 22
            new_max_pos = pos + length
            pos, v = packet2(inp, pos, new_max_pos, v=[])
        else:
            new_nr_sub = int(inp[pos+7:pos+18], 2)
            pos += 18
            pos, v = packet2(inp, pos, max_pos=0, nr_sub=new_nr_sub, v=[])    
        #print(typ, v)
        if typ == 0: v = [sum(v)]
        elif typ == 1:
            i = 1
            for j in v: i *= j
            v = [i]
        elif typ == 2: v = [min(v)]
        elif typ == 3: v = [max(v)]
        elif typ == 5: v = [int(v[0]>v[1])]
        elif typ == 6: v = [int(v[0]<v[1])]
        elif typ == 7: v = [int(v[0]==v[1])]   
        v = s+v
    if pos < max_pos: pos, v = packet2(inp, pos, max_pos, v=v)
    if nr_current < nr_sub: pos, v = packet2(inp, pos, max_pos=0, nr_sub=nr_sub, nr_current=nr_current+1, v=v)
        
    return pos, v

In [7]:
# 1 and 2
binary = to_binary(inp)
_, sol1 = packet1(binary, v=0)
_, sol2 = packet2(binary, v=[])
print(sol1)
print(sol2[0])

929
911945136934
